In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KDTree

def local_lda_predict(x0, X_train, y_train, bandwidth, reg=1e-3):
    diff = X_train - x0
    squared_dists = np.sum(diff**2, axis=1)
    weights = np.exp(-squared_dists / (2 * bandwidth**2)) 
    # gaussian kernel weights
    
    # esp to avoid div by zero errors
    eps = np.finfo(np.float64).eps
    total_weight = np.sum(weights) + eps
    
    # indicator matrix
    classes = np.unique(y_train)
    K = classes.shape[0]
    n_train, d = X_train.shape
    M = (y_train.reshape(-1, 1) == classes.reshape(1, -1)).astype(np.float64)
    
    # weighted sums
    weighted_sum = M.T @ (weights[:, None] * X_train)
    sum_w_per_class = M.T @ weights
    
    # weighted means per class w/ eps for div by zeero errors
    weighted_means = np.zeros((K, d))
    nonzero = sum_w_per_class > 0
    weighted_means[nonzero] = weighted_sum[nonzero] / sum_w_per_class[nonzero, None]
    
    # local priors
    class_priors = sum_w_per_class / total_weight
    
    # class means
    class_idx = np.searchsorted(classes, y_train)
    mu_per_sample = weighted_means[class_idx]
    
    # diff and cov matrix
    diff_per_sample = X_train - mu_per_sample
    weighted_diff = diff_per_sample * weights[:, None]
    cov = (weighted_diff.T @ weighted_diff) / total_weight
    cov += reg * np.eye(d)
    
    inv_cov = np.linalg.inv(cov)
    
    # discriminate scoring
    term1 = (weighted_means @ inv_cov.T) @ x0
    term2 = 0.5 * np.sum(weighted_means * (weighted_means @ inv_cov.T), axis=1)
    with np.errstate(divide='ignore'):
        term3 = np.where(class_priors > 0, np.log(class_priors), -np.inf)
    
    scores = term1 - term2 + term3
    best_class = classes[np.argmax(scores)]
    return best_class

def local_lda_predict_with_nn(x0, tree, X_train, y_train, 
bandwidth, k=50, reg=1e-3):
    _, ind = tree.query([x0], k=k)
    X_local = X_train[ind[0]]
    y_local = y_train[ind[0]]
    return local_lda_predict(x0, X_local, y_local, bandwidth, reg)

def local_lda_predict_many_with_nn(X_query, tree, X_train, 
y_train, bandwidth, k=50, reg=1e-3):
    n_query = X_query.shape[0]
    preds = np.empty(n_query, dtype=y_train.dtype)
    for i in range(n_query):
        preds[i] = local_lda_predict_with_nn(X_query[i], tree,
        X_train, y_train, bandwidth, k, reg)
    return preds

# since the dataset is not a friendly format...
train_df = pd.read_csv("zip.train", header=None, sep='\s+')
test_df = pd.read_csv("zip.test", header=None, sep='\s+')

# to limit dataset sizes (for debugging with faster run times, not for final runs)
train_df = train_df.iloc[:]
test_df = test_df.iloc[:]

# define the labels and features
y_train = train_df.iloc[:, 0].values
X_train = train_df.iloc[:, 1:].values
y_test  = test_df.iloc[:, 0].values
X_test  = test_df.iloc[:, 1:].values

#print(f"train shape: {X_train.shape} | test shape: {X_test.shape}") # debugging

# for k nearest neighbors
tree = KDTree(X_train)

# define params to sweep
bandwidth_values = [0.1, 0.2, 0.5, 0.75, 1.0]
results = []
k_neighbors = 50  # window size

for lam in bandwidth_values:
    
    y_train_pred = local_lda_predict_many_with_nn(X_train, 
    tree, X_train, y_train, lam, k=k_neighbors)
    train_error = np.mean(y_train_pred != y_train)
    
    y_test_pred = local_lda_predict_many_with_nn(X_test, 
    tree, X_train, y_train, lam, k=k_neighbors)
    test_error = np.mean(y_test_pred != y_test)
    
    print(f"Lambda: {lam} | Train: {train_error:.4f} | Test: {test_error:.4f}")
    
    results.append({
        "lambda": lam,
        "train_error": train_error,
        "test_error": test_error
    })